In [1]:
import random
import torch
import numpy as np


def setup_reproducibility(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(False, warn_only=True)
    torch.set_float32_matmul_precision("high")
    
SEED = 1000
setup_reproducibility(SEED)

In [2]:
from transformers import get_cosine_schedule_with_warmup
from scipy import signal
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from huggingface_hub import login, snapshot_download
from tqdm.auto import tqdm


def rest(t=4000):
    import time
    [time.sleep(1) for i in range(t)]
        
        
def generate_csv(preds, name):
    column_names = ['Glucose', 'Sodium Acetate', 'Magnesium Sulfate']
    preds_df = pd.DataFrame(preds, columns=column_names)
    preds_df.insert(0, 'ID', [i+1 for i in range(len(preds_df))])
    preds_df.to_csv(name, index=False)
    
    
def get_ckpt(path):
    return torch.load(path, weights_only=False)


def cuda_to_np(tensor):
    return tensor.cpu().detach().numpy()


def get_scheduler(optimizer, train_dl, epochs):
    total_training_steps = len(train_dl) * epochs
    warmup_steps = int(total_training_steps * 0.05)  # e.g. 5% warmup
    
    return get_cosine_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_training_steps
    )


def get_stats(tensor, p=True, r=False, minmax=False):
    if minmax:
        min, max = tensor.min(), tensor.max()
        mean, std = tensor.mean(), tensor.std()
        if p: print(f"Min: {min}, Max: {max} ,Mean: {mean}, Std: {std}")
        if r: return min, max, mean, std
    else:
        mean, std = tensor.mean(), tensor.std()
        if p: print(f"Mean: {mean}, Std: {std}")
        if r: return mean, std
    
    
def zscore(tensor, mean=None, std=None):
    if mean is None: mean = tensor.mean()
    if std is None: std = tensor.std()
    return (tensor - mean) / (std + 1e-8)


def reverse_zscore(tensor, mu, sigma):
    return (tensor * sigma) + mu


def get_model_size(model):
    print(sum(p.numel() for p in model.parameters()) / 1e6)
    

def get_index(iterable):
    return random.randint(0, len(iterable) - 1)


def get_indices(iterable, n):
    return random.sample(range(len(iterable)), n)


def split(inputs, targets, seed):
    return train_test_split(
        inputs,
        targets, 
        test_size=0.2,
        shuffle=True, 
        random_state=seed
    ) 


def show_waves(waves, dpi=100):
    """
    waves: numpy array of shape (3, N)
    Creates three separate figures that stretch wide.
    """
    N = waves.shape[1]
    t = np.arange(N)

    # Wide aspect ratio; height modest so each window fills width
    for i in range(waves.shape[0]):
        fig = plt.figure(figsize=(14, 4), dpi=dpi)  # wide figure
        ax = fig.add_subplot(111)
        ax.plot(t, waves[i], linewidth=1)
        ax.set_title(f"Wave {i+1}")
        ax.set_xlabel("Sample")
        ax.set_ylabel("Amplitude")
        ax.grid(True)
        fig.tight_layout()  # reduce margins to use width
        
    plt.show()
    
    
def hf_ds_download(hf_token, repo_id):
    login(hf_token[1:])
    return snapshot_download(repo_id, repo_type="dataset")


def get_spectra_features(X, b=False):
    """Create multi-channel features from spectra: raw, 1st derivative, 2nd derivative."""
    X_processed = np.zeros_like(X)
    # Baseline correction and SNV
    for i in tqdm(range(X.shape[0])):
        poly = np.polyfit(np.arange(X.shape[1]), X[i], 3)
        baseline = np.polyval(poly, np.arange(X.shape[1]))
        corrected_spec = X[i] - baseline
        #X_processed[i] = (corrected_spec - corrected_spec.mean()) / (corrected_spec.std() + 1e-8)
        X_processed[i] = corrected_spec
        
    # Calculate derivatives
    deriv1 = signal.savgol_filter(X_processed, window_length=11, polyorder=3, deriv=1, axis=1)
    deriv2 = signal.savgol_filter(X_processed, window_length=11, polyorder=3, deriv=2, axis=1)

    if b: return np.stack([X_processed, deriv1, deriv2], axis=1)
    return np.stack([deriv1, deriv2], axis=1)

In [3]:
import os

if False:
    path = "/kaggle/input/dig-4-bio-raman-transfer-learning-challenge"
    files = os.listdir(path)
    [(i, files[i]) for i in range(len(files))]

In [4]:
hf_token = "xhf_XURkoNhwOIPtEdHfNeRpVkjEwKSkhtigFi"
path = hf_ds_download(hf_token, "ArbaazBeg/kaggle-spectogram")
files = os.listdir(path)
[(i, files[i]) for i in range(len(files))]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

[(0, '.gitattributes'),
 (1, 'sample_submission.csv'),
 (2, 'anton_785.csv'),
 (3, 'anton_532.csv'),
 (4, '96_samples.csv'),
 (5, 'metrohm.csv'),
 (6, 'timegate.csv'),
 (7, 'kaiser.csv'),
 (8, 'transfer_plate.csv'),
 (9, 'mettler_toledo.csv'),
 (10, 'tec5.csv'),
 (11, 'tornado.csv')]

In [5]:
def load_test_data():
    test = pd.read_csv(os.path.join(path, files[4]))

    row1 = test.columns[1:].to_numpy().copy()
    row1[-1] = "5611"
    row1 = row1.astype(np.float64)


    cols = test.columns[1:]
    test = test[cols]
    test[" 5611]"] = test[" 5611]"].str.replace('[\[\]]', '', regex=True).astype('int64')
    test = test.to_numpy()

    test = np.insert(test, 0, row1, axis=0)
    return test.reshape(-1, 2, 2048).mean(axis=1)


def get_test_data():
    inputs = load_test_data()
    
    spectra_selection = np.logical_and(
        300 <= np.array([float(one) for one in range(2048)]),
        np.array([float(one) for one in range(2048)]) <= 1942,
    )
    
    inputs = inputs[:, spectra_selection]

    wns = np.array([
        float(one) for one in range(2048)
    ])[spectra_selection]
    wavenumbers = np.arange(300, 1943)

    interpolated_data = np.array(
        [np.interp(wavenumbers, xp=wns, fp=i) for i in inputs]
    )

    normed_spectra = interpolated_data / np.max(interpolated_data)
    return normed_spectra
    

<>:11: SyntaxWarning: invalid escape sequence '\['
<>:11: SyntaxWarning: invalid escape sequence '\['
/tmp/ipykernel_132415/3281430402.py:11: SyntaxWarning: invalid escape sequence '\['
  test[" 5611]"] = test[" 5611]"].str.replace('[\[\]]', '', regex=True).astype('int64')


In [6]:
import numpy as np
import pandas as pd


def load_transfer_data():
    csv_path = os.path.join(path, files[8])
    df = pd.read_csv(csv_path)

    input_cols = df.columns[1:2049]
    target_cols = df.columns[2050:]

    targets  = df[target_cols].dropna().to_numpy()

    df = df[input_cols]
    df['Unnamed: 1'] = df['Unnamed: 1'].str.replace("[\[\]]", "", regex=True).astype('int64')
    df['Unnamed: 2048'] = df['Unnamed: 2048'].str.replace("[\[\]]", "", regex=True).astype('int64')

    inputs = df.to_numpy().reshape(-1, 2, 2048)
    inputs = inputs.mean(axis=1)
    
    return inputs, targets


def preprocess_transfer_data():
    inputs, targets = load_transfer_data()
    
    spectra_selection = np.logical_and(
        300 <= np.array([float(one) for one in range(2048)]),
        np.array([float(one) for one in range(2048)]) <= 1942,
    )
    
    inputs = inputs[:, spectra_selection]
    
    wns = np.array([
        float(one) for one in range(2048)
    ])[spectra_selection]
    wavenumbers = np.arange(300, 1943)
    
    interpolated_data = np.array(
        [np.interp(wavenumbers, xp=wns, fp=i) for i in inputs]
    )
    
    normed_spectra = interpolated_data / np.max(interpolated_data)
    return normed_spectra, targets

inputs, targets = preprocess_transfer_data()
inputs.shape, targets.shape

<>:15: SyntaxWarning: invalid escape sequence '\['
<>:16: SyntaxWarning: invalid escape sequence '\['
<>:15: SyntaxWarning: invalid escape sequence '\['
<>:16: SyntaxWarning: invalid escape sequence '\['
/tmp/ipykernel_132415/1956701579.py:15: SyntaxWarning: invalid escape sequence '\['
  df['Unnamed: 1'] = df['Unnamed: 1'].str.replace("[\[\]]", "", regex=True).astype('int64')
/tmp/ipykernel_132415/1956701579.py:16: SyntaxWarning: invalid escape sequence '\['
  df['Unnamed: 2048'] = df['Unnamed: 2048'].str.replace("[\[\]]", "", regex=True).astype('int64')


((96, 1643), (96, 3))

In [7]:
import random
import torch
from torch.utils.data import Dataset
import scipy.optimize


np_dtype_from_torch = {
    torch.float32: np.float32,
    torch.float64: np.float64,
}

class SpectralDataset(Dataset):
    def __init__(
        self,
        spectra,
        concentrations,
        dtype=None,
        spectra_mean_std=None,
        concentration_mean_std=None,
        combine_spectra_range=0.0,
        baseline_factor_bound=0.0,
        baseline_period_lower_bound=100.0,
        baseline_period_upper_bound=200.0,
        augment_slope_std=0.0,
        augment_intersept_std=0.0,
        rolling_bound=0,
        spectrum_rolling_sigma=0.0,
        augmentation_weight=0.1,
        original_datapoint_weight=1.,
    ):
        self.dtype = dtype or torch.float32
        self.combine_spectra_range = combine_spectra_range
        self.baseline_factor_bound = baseline_factor_bound
        self.augment_slope_std = augment_slope_std
        self.augment_intercept_std = augment_intersept_std
        self.baseline_period_lower_bound = baseline_period_lower_bound
        self.baseline_period_upper_bound = baseline_period_upper_bound
        self.rolling_bound = rolling_bound
        self.spectrum_rolling_sigma = spectrum_rolling_sigma
        self.augmentation_weight = torch.tensor(augmentation_weight, dtype=dtype)
        self.original_dp_weight = original_datapoint_weight

        # normalize spectra
        spectra = torch.tensor(spectra, dtype=dtype)

        if spectra_mean_std is None:
            self.s_mean = torch.mean(spectra)
            self.s_std = torch.std(spectra)
        else:
            self.s_mean, self.s_std = spectra_mean_std

        self.spectra = torch.divide(
            torch.subtract(spectra, self.s_mean),
            self.s_std,
        )

        self.dummy_wns = np.tile(
            np.arange(
                0., 1., 1. / self.spectra.shape[2],
                dtype=np_dtype_from_torch[self.dtype]
            )[None, :self.spectra.shape[2]],
            (self.spectra.shape[1], 1),
        )

        # normalize concentrations
        concentrations = torch.tensor(concentrations, dtype=dtype)
        if concentration_mean_std is None:
            self.concentration_means = torch.nanmean(concentrations, dim=0)

            self.concentration_stds = torch.maximum(
                torch.tensor(
                    [
                        torch.std(col[torch.logical_not(torch.isnan(col))])
                        for col in concentrations.T
                    ]
                ),
                torch.tensor([1e-3] * concentrations.shape[1]),
            )
        else:
            self.concentration_means = concentration_mean_std[0]
            self.concentration_stds = concentration_mean_std[1]

        self.concentrations = torch.divide(
            torch.subtract(
                concentrations,
                self.concentration_means,
            ),
            self.concentration_stds,
        )

    def pick_two(self, max_idx=None):
        max_idx = max_idx or len(self)
        return random.choices(range(max_idx), k=2)

    def __len__(self):
        return len(self.concentrations)

    def augment_spectra(self, spectra):
        if self.augment_slope_std > 0.0:

            def spectrum_approximation(x, slope, intercept):
                return (slope * x + intercept).reshape(-1, 1)[:, 0]

            slope, inter = scipy.optimize.curve_fit(
                spectrum_approximation,
                self.dummy_wns,
                spectra.reshape(-1, 1)[:, 0],
                p0=np.random.rand(2),
            )[0]

            new_slope = slope * (
                    np.random.gamma(
                        shape=1. / self.augment_slope_std,
                        scale=self.augment_slope_std,
                        size=1,
                    )
            )[0]
            new_intercept = inter * (
                1.0 + np.random.randn(1) * self.augment_intercept_std
            )[0]
            spectra += torch.tensor(
                (new_slope - slope)
            ) * self.dummy_wns + new_intercept - inter

        factor = self.baseline_factor_bound * torch.rand(size=(1,))
        offset = torch.rand(size=(1,)) * 2.0 * torch.pi
        period = self.baseline_period_lower_bound + (
            self.baseline_period_upper_bound - self.baseline_period_lower_bound
        ) * torch.rand(size=(1,))
        permutations = factor * torch.cos(
            2.0 * torch.pi / period * self.dummy_wns + offset
        )
        return self.roll_spectrum(
            spectra + permutations * spectra,
            delta=random.randint(-self.rolling_bound, self.rolling_bound),
        )

    def roll_spectrum(self, spectra, delta):
        num_spectra = spectra.shape[0]
        rolled_spectra = np.roll(spectra, delta, axis=1)
        if delta > 0:
            rolled_spectra[:, :delta] = (
                np.random.rand(num_spectra, delta) * self.spectrum_rolling_sigma + 1
            ) * rolled_spectra[:, delta:(delta + 1)]
        elif delta < 0:
            rolled_spectra[:, delta:] = (
                np.random.rand(num_spectra, -delta) * self.spectrum_rolling_sigma + 1
            ) * rolled_spectra[:, delta - 1:delta]
        return rolled_spectra

    def combine_k_items(self, indices, weights):
        return (
            # spectra
            torch.sum(
                torch.mul(weights[:, None, None], self.spectra[indices, :, :]),
                dim=0,
            ),
            # concentrations
            torch.sum(
                torch.mul(weights[:, None], self.concentrations[indices, :]),
                dim=0,
            )
        )

    def __getitem__(self, idx):
        if self.combine_spectra_range < 1e-12:
            spectrum = self.spectra[idx]
            spectrum = self.augment_spectra(spectrum)
            return {
                "spectra": spectrum,
                "concentrations": self.concentrations[idx],
                "label_weight": torch.tensor(1.0, dtype=self.dtype),
            }
        else:
            if random.random() < self.original_dp_weight:
                one_weight = 1.
                label_weight = torch.tensor(1.0, dtype=self.dtype)
            else:
                one_weight = random.uniform(0.0, self.combine_spectra_range)
                label_weight = self.augmentation_weight
            weights = torch.tensor([one_weight, (1 - one_weight)])
            # just pick two random indices
            indices = random.choices(range(len(self)), k=2)

            mixed_spectra, mixed_concentrations = self.combine_k_items(
                indices=indices,
                weights=weights,
            )
            mixed_spectra = self.augment_spectra(mixed_spectra)
            return mixed_spectra, mixed_concentrations, label_weight


config = {
    'initial_cnn_channels': 32,
    'cnn_channel_factor': 1.279574024454846,
    'num_cnn_layers': 8,
    'kernel_size': 3,
    'stride': 2,
    'activation_function': 'ELU',
    'fc_dropout': 0.10361700399831791,
    'lr': 0.001,
    'gamma': 0.9649606352621118,
    'baseline_factor_bound': 0.748262317340447,
    'baseline_period_lower_bound': 0.9703081695287203,
    'baseline_period_span': 19.79744237606427,
    'original_datapoint_weight': 0.4335003268130408,
    'augment_slope_std': 0.08171025264382692,
    'batch_size': 32,
    'fc_dims': 226,
    'rolling_bound': 2,
    'num_blocks': 2,
}

def get_dataset(inputs, targets, config, inputs_mean_std=None, targets_mean_std=None):
    return SpectralDataset(
        spectra=inputs[:, None, :],
        concentrations=targets,
        dtype=torch.float32,
        spectra_mean_std=inputs_mean_std,
        concentration_mean_std=targets_mean_std,
        combine_spectra_range=1.0,
        baseline_factor_bound=config["baseline_factor_bound"],
        baseline_period_lower_bound=config["baseline_period_lower_bound"],
        baseline_period_upper_bound=(config["baseline_period_lower_bound"] + config["baseline_period_span"]),
        augment_slope_std=config["augment_slope_std"],
        augment_intersept_std=0.0,
        rolling_bound=config["rolling_bound"],
        spectrum_rolling_sigma=0.01,
        augmentation_weight=0.1,
        original_datapoint_weight=1.,
    )

In [8]:
from torch.utils.data import DataLoader


def build_loader(
    SEED,
    ds,
    train=True,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    drop_last=True,
    pin_memory=True,
    persistent_workers=False,
):
    def seed_worker(worker_id):
        worker_seed = torch.initial_seed() % 2**32
        np.random.seed(worker_seed)
        random.seed(worker_seed)

    generator = torch.Generator()
    generator.manual_seed(SEED if train else SEED+5232)

    return DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=pin_memory,
        drop_last=drop_last,
        persistent_workers=persistent_workers,
        worker_init_fn=seed_worker,
        generator=generator,
        #sampler=DistributedSampler(
        #    train_ds,
        #    shuffle=True,
        #    drop_last=True,
        #    seed=config.seed
        #)
    )
    
    
def return_dls(train_ds, eval_ds, train_batch_size, eval_batch_size):
    train_dl = build_loader(
        SEED,
        train_ds,
        train=True,
        batch_size=train_batch_size,
        shuffle=True,
        num_workers=0,
        drop_last=False,
        pin_memory=True,
        persistent_workers=False,
    )

    eval_dl = build_loader(
        SEED,
        eval_ds,
        train=False,
        batch_size=eval_batch_size,
        shuffle=False,
        num_workers=0,
        drop_last=False,
        pin_memory=True,
        persistent_workers=False,
    )
    
    return train_dl, eval_dl

In [9]:
import neptune


def setup_neptune():
    if not RESUME:
        neptune_run = neptune.init_run(
            project="arbaaz/kaggle-spect",
            name=MODEL_NAME,
            api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJlOGE2YjNiZS1mZGUyLTRjYjItYTg5Yy1mZWJkZTIzNzE1NmIifQ=="
        )

        neptune_run["h_parameters"] = {
            "seed": SEED,
            "model_name": MODEL_NAME,
            "optimizer_name": "nadam",
            "learning_rate": LR,
            "scheduler_name": "default",
            "weight_decay": WD,
            "num_epochs": EPOCHS,
            "batch_size": BATCH_SIZE,
        }
        if DROPOUT: neptune_run["h_parameters"] = {"dropout": DROPOUT}
        if DROP_PATH_RATE: neptune_run["h_parameters"] = {"drop_path_rate": DROP_PATH_RATE}
    else:
        neptune_run = neptune.init_run(
            project="arbaaz/crunchdao-structural-break",
            with_id=config.with_id,
            api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJlOGE2YjNiZS1mZGUyLTRjYjItYTg5Yy1mZWJkZTIzNzE1NmIifQ=="
        )

    return neptune_run

In [10]:
import torch.nn.functional as F
from torch.nn.modules.loss import _Loss
from sklearn.metrics import r2_score


def loss_fn(logits, targets):
    logits = logits.view(-1)
    targets = targets.view(-1)
    return F.mse_loss(logits, targets)


def metric_fn(logits, targets):
    preds = logits.cpu().detach().float().numpy()
    targets = targets.cpu().detach().numpy()
    
    dim1 = r2_score(targets[:, 0], preds[:, 0])
    dim2 = r2_score(targets[:, 1], preds[:, 1])
    dim3 = r2_score(targets[:, 2], preds[:, 2])
    
    return dim1, dim2, dim3, r2_score(targets, preds)


class MSEIgnoreNans(_Loss):
    def forward(
        self,
        input: torch.Tensor,
        target: torch.Tensor,
        weights: torch.Tensor,
    ) -> torch.Tensor:
        mask = torch.isfinite(target)
        mse = torch.mean(
            torch.mul(
                torch.square(input[mask] - target[mask]),
                torch.tile(weights[:, None], dims=(1, target.shape[1]))[mask],
            )
        )
        return torch.where(
            torch.isfinite(mse),
            mse,
            torch.tensor(0.).to(target.device),
        )

In [11]:
import math


class Identity(torch.torch.nn.Module):
    def forward(self, x):
        return x


# this is not a resnet yet
class ReZeroBlock(torch.torch.nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        activation_function,
        kernel_size,
        stride,
        dtype,
        norm_layer=None,
    ):
        super(ReZeroBlock, self).__init__()
        if norm_layer is None:
            norm_layer = torch.torch.nn.BatchNorm1d

        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = divmod(kernel_size, 2)[0] if stride == 1 else 0

        # does not change spatial dimension
        self.conv1 = torch.nn.Conv1d(
            in_channels,
            out_channels,
            kernel_size=1,
            stride=1,
            bias=False,
            dtype=dtype,
        )
        self.bn1 = norm_layer(out_channels, dtype=dtype)
        # Both self.conv2 and self.downsample layers
        # downsample the input when stride != 1
        self.conv2 = torch.nn.Conv1d(
            out_channels,
            out_channels,
            kernel_size=kernel_size,
            stride=stride,
            groups=out_channels,
            bias=False,
            dtype=dtype,
            padding=self.padding,
        )
        if stride > 1:
            down_conv = torch.nn.Conv1d(
                in_channels,
                out_channels,
                kernel_size=kernel_size,
                stride=stride,
                bias=False,
                dtype=dtype,
                # groups=out_channels,
            )
        else:
            down_conv = Identity()

        self.down_sample = torch.nn.Sequential(
            down_conv,
            norm_layer(out_channels),
        )
        self.bn2 = norm_layer(out_channels, dtype=dtype)
        # does not change the spatial dimension
        self.conv3 = torch.nn.Conv1d(
            out_channels,
            out_channels,
            kernel_size=1,
            stride=1,
            bias=False,
            dtype=dtype,
        )
        self.bn3 = norm_layer(out_channels, dtype=dtype)
        self.activation = activation_function(inplace=True)
        self.factor = torch.torch.nn.parameter.Parameter(torch.tensor(0.0, dtype=dtype))

    def next_spatial_dim(self, last_spatial_dim):
        return math.floor(
            (last_spatial_dim + 2 * self.padding - self.kernel_size)
            / self.stride + 1
        )

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.activation(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.activation(out)

        out = self.conv3(out)
        out = self.bn3(out)

        # not really the identity, but kind of
        identity = self.down_sample(x)

        return self.activation(out * self.factor + identity)


class ResNetEncoder(torch.torch.nn.Module):
    def __init__(
        self,
        spectrum_size,
        cnn_encoder_channel_dims,
        activation_function,
        kernel_size,
        stride,
        dtype,
        num_blocks,
        verbose=False,
    ):
        super(ResNetEncoder, self).__init__()

        self.spatial_dims = [spectrum_size]
        layers = []
        for in_channels, out_channels in zip(
            cnn_encoder_channel_dims[:-1],
            cnn_encoder_channel_dims[1:],
        ):
            block = ReZeroBlock(
                in_channels=in_channels,
                out_channels=out_channels,
                activation_function=activation_function,
                kernel_size=kernel_size,
                stride=stride,
                dtype=dtype,
            )
            layers.append(block)
            self.spatial_dims.append(block.next_spatial_dim(self.spatial_dims[-1]))
            for _ in range(num_blocks - 1):
                block = ReZeroBlock(
                    in_channels=out_channels,
                    out_channels=out_channels,
                    activation_function=activation_function,
                    kernel_size=kernel_size,
                    stride=1,
                    dtype=dtype,
                )
                layers.append(block)
                self.spatial_dims.append(block.next_spatial_dim(self.spatial_dims[-1]))

        self.resnet_layers = torch.torch.nn.Sequential(*layers)
        if verbose:
            print("CNN Encoder Channel Dims: %s" % (cnn_encoder_channel_dims))
            print("CNN Encoder Spatial Dims: %s" % (self.spatial_dims))

    def forward(self, x):
        return self.resnet_layers(x)


class ReZeroNet(torch.nn.Module):
    def __init__(
        self,
        spectra_channels,
        spectra_size,
        initial_cnn_channels,
        cnn_channel_factor,
        num_cnn_layers,
        kernel_size,
        stride,
        activation_function,
        fc_dims,
        fc_dropout=0.0,
        dtype=None,
        verbose=False,
        fc_output_channels=1,
        num_blocks=1,
        **kwargs,
    ):
        super().__init__()
        self.fc_output_channels = fc_output_channels
        self.dtype = dtype or torch.float32

        activation_function = getattr(torch.nn, activation_function)

        # Setup CNN Encoder
        cnn_encoder_channel_dims = [spectra_channels] + [
            int(initial_cnn_channels * (cnn_channel_factor**idx))
            for idx in range(num_cnn_layers)
        ]
        self.cnn_encoder = ResNetEncoder(
            spectrum_size=spectra_size,
            cnn_encoder_channel_dims=cnn_encoder_channel_dims,
            activation_function=activation_function,
            kernel_size=kernel_size,
            stride=stride,
            num_blocks=num_blocks,
            dtype=dtype,
            verbose=verbose,
        )
        self.fc_dims = [
            int(
                self.cnn_encoder.spatial_dims[-1]
            ) * int(cnn_encoder_channel_dims[-1])
        ] + fc_dims

        if verbose:
            print("Fc Dims: %s" % self.fc_dims)
        fc_layers = []
        for idx, (in_dim, out_dim) in enumerate(
                zip(self.fc_dims[:-2], self.fc_dims[1:-1])
        ):
            fc_layers.append(torch.nn.Linear(in_dim, out_dim))
            fc_layers.append(torch.nn.ELU())
            fc_layers.append(torch.nn.Dropout(fc_dropout / (2 ** idx)))
        fc_layers.append(
            torch.nn.Linear(
                self.fc_dims[-2],
                self.fc_dims[-1] * self.fc_output_channels,
            ),
        )
        self.fc_net = torch.nn.Sequential(*fc_layers)
        if verbose:
            num_params = sum(p.numel() for p in self.parameters())
            print("Number of Parameters: %s" % num_params)

    def forward(self, spectra):
        embeddings = self.cnn_encoder(spectra)
        forecast = self.fc_net(embeddings.view(-1, self.fc_dims[0]))
        if self.fc_output_channels > 1:
            forecast = forecast.reshape(
                -1, self.fc_output_channels, self.fc_dims[-1]
            )
        return forecast


In [12]:
from tqdm.auto import tqdm


mse_loss_function = MSEIgnoreNans()

def train(
    model, 
    optimizer,
    device,
    scaler, 
    scheduler,
    train_dl,
    eval_dl,
    loss_fn,
    epochs,
    checkpoint_name,
    score=-float("inf"),
    neptune_run=None,
    p=True,
):  
    for epoch in tqdm(range(epochs)):
        model.train()
        total_loss = 0.0
        all_logits = []
        all_targets = []
        
        for inputs, targets, weights in train_dl:
            inputs = inputs.to(device, non_blocking=True)
            targets = targets.to(device, non_blocking=True)
            weights = weights.to(device, non_blocking=True)
            
            optimizer.zero_grad()
            with torch.amp.autocast(device_type=device, dtype=torch.bfloat16, cache_enabled=True):
                logits = model(inputs)
                loss = loss_fn(logits, targets)
                            
            loss.backward()
            optimizer.step()
            #scaler.scale(loss).backward()
            #scaler.step(optimizer)
            #scaler.update()
            scheduler.step()
            if neptune_run is not None:  neptune_run["lr_step"].append(scheduler.get_last_lr()[0])
            
            total_loss += loss.detach().cpu()
            all_logits.append(logits.detach().cpu())
            all_targets.append(targets.detach().cpu())
        
        all_logits = torch.cat(all_logits)
        all_targets = torch.cat(all_targets)

        one, two, three, r2 = metric_fn(all_logits, all_targets)
        total_loss = total_loss / len(train_dl)
        
        model.eval()
        eval_total_loss = 0.0
        eval_all_logits = []
        eval_all_targets = []

        for inputs, targets, weights in eval_dl:
            inputs = inputs.to(device, non_blocking=True)
            targets = targets.to(device, non_blocking=True)
            weights = weights.to(device, non_blocking=True)

            with torch.inference_mode():
                with torch.amp.autocast(device_type=device, dtype=torch.bfloat16, cache_enabled=True):
                    logits = model(inputs)
                    loss = loss_fn(logits, targets)

            eval_total_loss += loss.detach().cpu()
            eval_all_logits.append(logits.detach().cpu())
            eval_all_targets.append(targets.detach().cpu())
        
        eval_all_logits = torch.cat(eval_all_logits)
        eval_all_targets = torch.cat(eval_all_targets)

        eval_one, eval_two, eval_three, eval_r2 = metric_fn(eval_all_logits, eval_all_targets)
        eval_total_loss = eval_total_loss / len(eval_dl)
        
        if eval_r2 > score:
            score = eval_r2
            data = {"state_dict": model.state_dict()}
            data["epoch"] = epoch 
            data["score"] = score
            torch.save(data, f"/{checkpoint_name}")
        
        if neptune_run is not None:
            neptune_run["train/loss"].append(total_loss)
            neptune_run["eval/loss"].append(eval_total_loss)
            neptune_run["train/r2"].append(r2)
            neptune_run["eval/r2"].append(eval_r2)
            neptune_run["train/one"].append(one)
            neptune_run["train/two"].append(two)
            neptune_run["train/three"].append(three)
            neptune_run["eval/one"].append(eval_one)
            neptune_run["eval/two"].append(eval_two)
            neptune_run["eval/three"].append(eval_three)
            
        if p and epoch % 5 == 0:
            print(
                f"Epoch: {epoch}, "
                f"train/loss: {total_loss:.4f}, "
                f"eval/loss: {eval_total_loss:.4f}, "
                f"train/r2: {r2:.4f}, "
                f"eval/r2: {eval_r2:.4f}, "
                f"train/one: {one:.4f}, "
                f"train/two: {two:.4f}, "
                f"train/three: {three:.4f}, "
                f"eval/one: {eval_one:.4f}, "
                f"eval/two: {eval_two:.4f}, "
                f"eval/three: {eval_three:.4f} "
            )
            
    if neptune_run is not None: neptune_run.stop()
    return score

In [13]:
import warnings#; warnings.filterwarnings("ignore")


EPOCHS = 100
WD = 1e-3
LR = 1e-4

DROPOUT = 0.5
DROP_PATH_RATE = None

device = "cuda" if torch.cuda.is_available() else "cpu"
RESUME = False

config["dtype"] = torch.float32
config["spectra_size"] = 1643
config["spectra_channels"] = 1
config["fc_dims"] = [
    config["fc_dims"],
    int(config["fc_dims"] / 2),
    3,
]

#mse_loss_function = MSEIgnoreNans()

In [14]:
from sklearn.model_selection import KFold


inputs_mean_std = []
targets_mean_std = []
scores = []
kfold = KFold(n_splits=5, shuffle=True, random_state=SEED)
splits = kfold.split(inputs)

for fold, (train_idx, eval_idx) in enumerate(splits):
    MODEL_NAME = f"resnet.paper.pretrain.avg18.fold.{fold}"
    checkpoint_name = f"paper.pretrain.avg18.fold.{fold}.pt"
    
    train_inputs = inputs[train_idx]
    train_targets = targets[train_idx]
    eval_inputs = inputs[eval_idx]
    eval_targets = targets[eval_idx]

    train_ds = get_dataset(train_inputs, train_targets, config)
    
    inputs_mean_std.append((fold, train_ds.s_mean, train_ds.s_std))
    targets_mean_std.append((fold, train_ds.concentration_means, train_ds.concentration_stds))
    
    eval_ds = get_dataset(eval_inputs, eval_targets, config, (train_ds.s_mean, train_ds.s_std), (train_ds.concentration_means, train_ds.concentration_stds))
    
    BATCH_SIZE = 32
    train_dl, eval_dl = return_dls(train_ds, eval_ds, BATCH_SIZE, len(eval_ds))
    
    #model = ResNet(input_channels=1, dropout=DROPOUT).to(device)
    model = ReZeroNet(**config).to(device)
    if fold == 0: print(get_model_size(model))
    
    ckpt = get_ckpt("/paper.pretrain.avg.pt")
    model.load_state_dict(ckpt)#["state_dict"])
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD, foreach=True)
    scaler = torch.amp.GradScaler(device)
    scheduler = get_scheduler(optimizer, train_dl, EPOCHS)
    
    score = train(
            model, 
            optimizer, 
            device,
            scaler,
            scheduler,
            train_dl, 
            eval_dl,
            loss_fn,
            EPOCHS,
            checkpoint_name,
            neptune_run=setup_neptune(),
        )
    
    scores.append(score)

0.734309
None


[neptune] [warning] NeptuneWarning: By default, these monitoring options are disabled in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', 'capture_hardware_metrics'. You can set them to 'True' when initializing the run and the monitoring will continue until you call run.stop() or the kernel stops. NOTE: To track the source files, pass their paths to the 'source_code' argument. For help, see: https://docs-legacy.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/arbaaz/kaggle-spect/e/KAG-193


  0%|          | 0/100 [00:00<?, ?it/s]

/tmp/ipykernel_132415/489186478.py:121: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  spectra += torch.tensor(
/tmp/ipykernel_132415/489186478.py:131: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  2.0 * torch.pi / period * self.dummy_wns + offset
/tmp/ipykernel_132415/489186478.py:121: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  spectra += torch.tensor(
/tmp/ipykernel_132415/489186478.py:131: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  2.0 * torch.pi / period * self.dummy_wns + offset
/tmp/ipykernel_132415/489186478.py:121: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) 

...

Epoch: 95, train/loss: 0.3776, eval/loss: 0.3053, train/r2: 0.6102, eval/r2: 0.4709, train/one: 0.7888, train/two: 0.2080, train/three: 0.8337, eval/one: 0.8329, eval/two: -0.1607, eval/three: 0.7406 


/tmp/ipykernel_132415/489186478.py:121: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  spectra += torch.tensor(
/tmp/ipykernel_132415/489186478.py:131: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  2.0 * torch.pi / period * self.dummy_wns + offset
/tmp/ipykernel_132415/489186478.py:121: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  spectra += torch.tensor(
/tmp/ipykernel_132415/489186478.py:131: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  2.0 * torch.pi / period * self.dummy_wns + offset
/tmp/ipykernel_132415/489186478.py:121: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) 

[neptune] [info   ] Shutting down background jobs, please wait a moment...


/tmp/ipykernel_132415/489186478.py:121: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  spectra += torch.tensor(
/tmp/ipykernel_132415/489186478.py:131: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  2.0 * torch.pi / period * self.dummy_wns + offset


[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 127 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 127 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/arbaaz/kaggle-spect/e/KAG-197/metadata


In [14]:
class SpectralTestDataset(Dataset):
    def __init__(
        self,
        spectra,
        concentrations,
        dtype=None,
        spectra_mean_std=None,
        concentration_mean_std=None,
        combine_spectra_range=0.0,
        baseline_factor_bound=0.0,
        baseline_period_lower_bound=100.0,
        baseline_period_upper_bound=200.0,
        augment_slope_std=0.0,
        augment_intersept_std=0.0,
        rolling_bound=0,
        spectrum_rolling_sigma=0.0,
        augmentation_weight=0.1,
        original_datapoint_weight=1.,
    ):
        self.dtype = dtype or torch.float32
        self.combine_spectra_range = combine_spectra_range
        self.baseline_factor_bound = baseline_factor_bound
        self.augment_slope_std = augment_slope_std
        self.augment_intercept_std = augment_intersept_std
        self.baseline_period_lower_bound = baseline_period_lower_bound
        self.baseline_period_upper_bound = baseline_period_upper_bound
        self.rolling_bound = rolling_bound
        self.spectrum_rolling_sigma = spectrum_rolling_sigma
        self.augmentation_weight = torch.tensor(augmentation_weight, dtype=dtype)
        self.original_dp_weight = original_datapoint_weight

        # normalize spectra
        spectra = torch.tensor(spectra, dtype=dtype)

        if spectra_mean_std is None:
            self.s_mean = torch.mean(spectra)
            self.s_std = torch.std(spectra)
        else:
            self.s_mean, self.s_std = spectra_mean_std

        self.spectra = torch.divide(
            torch.subtract(spectra, self.s_mean),
            self.s_std,
        )

        self.dummy_wns = np.tile(
            np.arange(
                0., 1., 1. / self.spectra.shape[2],
                dtype=np_dtype_from_torch[self.dtype]
            )[None, :self.spectra.shape[2]],
            (self.spectra.shape[1], 1),
        )

        if False:
            # normalize concentrations
            concentrations = torch.tensor(concentrations, dtype=dtype)
            if concentration_mean_std is None:
                self.concentration_means = torch.nanmean(concentrations, dim=0)

                self.concentration_stds = torch.maximum(
                    torch.tensor(
                        [
                            torch.std(col[torch.logical_not(torch.isnan(col))])
                            for col in concentrations.T
                        ]
                    ),
                    torch.tensor([1e-3] * concentrations.shape[1]),
                )
            else:
                self.concentration_means = concentration_mean_std[0]
                self.concentration_stds = concentration_mean_std[1]

            self.concentrations = torch.divide(
                torch.subtract(
                    concentrations,
                    self.concentration_means,
                ),
                self.concentration_stds,
            )

    def pick_two(self, max_idx=None):
        max_idx = max_idx or len(self)
        return random.choices(range(max_idx), k=2)

    def __len__(self):
        return 96

    def augment_spectra(self, spectra):
        if self.augment_slope_std > 0.0:

            def spectrum_approximation(x, slope, intercept):
                return (slope * x + intercept).reshape(-1, 1)[:, 0]

            slope, inter = scipy.optimize.curve_fit(
                spectrum_approximation,
                self.dummy_wns,
                spectra.reshape(-1, 1)[:, 0],
                p0=np.random.rand(2),
            )[0]

            new_slope = slope * (
                    np.random.gamma(
                        shape=1. / self.augment_slope_std,
                        scale=self.augment_slope_std,
                        size=1,
                    )
            )[0]
            new_intercept = inter * (
                1.0 + np.random.randn(1) * self.augment_intercept_std
            )[0]
            spectra += torch.tensor(
                (new_slope - slope)
            ) * self.dummy_wns + new_intercept - inter

        factor = self.baseline_factor_bound * torch.rand(size=(1,))
        offset = torch.rand(size=(1,)) * 2.0 * torch.pi
        period = self.baseline_period_lower_bound + (
            self.baseline_period_upper_bound - self.baseline_period_lower_bound
        ) * torch.rand(size=(1,))
        permutations = factor * torch.cos(
            2.0 * torch.pi / period * self.dummy_wns + offset
        )
        return self.roll_spectrum(
            spectra + permutations * spectra,
            delta=random.randint(-self.rolling_bound, self.rolling_bound),
        )

    def roll_spectrum(self, spectra, delta):
        num_spectra = spectra.shape[0]
        rolled_spectra = np.roll(spectra, delta, axis=1)
        if delta > 0:
            rolled_spectra[:, :delta] = (
                np.random.rand(num_spectra, delta) * self.spectrum_rolling_sigma + 1
            ) * rolled_spectra[:, delta:(delta + 1)]
        elif delta < 0:
            rolled_spectra[:, delta:] = (
                np.random.rand(num_spectra, -delta) * self.spectrum_rolling_sigma + 1
            ) * rolled_spectra[:, delta - 1:delta]
        return rolled_spectra

    def combine_k_items(self, indices, weights):
        return (
            # spectra
            torch.sum(
                torch.mul(weights[:, None, None], self.spectra[indices, :, :]),
                dim=0,
            ),
            # concentrations
            #torch.sum(
            #    torch.mul(weights[:, None], self.concentrations[indices, :]),
            #    dim=0,
            #)
        )

    def __getitem__(self, idx):
        if True:#self.combine_spectra_range < 1e-12:
            spectrum = self.spectra[idx]
            #spectrum = self.augment_spectra(spectrum)
            return spectrum
        else:
            if random.random() < self.original_dp_weight:
                one_weight = 1.
                label_weight = torch.tensor(1.0, dtype=self.dtype)
            else:
                one_weight = random.uniform(0.0, self.combine_spectra_range)
                label_weight = self.augmentation_weight
            weights = torch.tensor([one_weight, (1 - one_weight)])
            # just pick two random indices
            indices = random.choices(range(len(self)), k=2)

            mixed_spectra = self.combine_k_items(
                indices=indices,
                weights=weights,
            )
            mixed_spectra = self.augment_spectra(mixed_spectra[0])
            return mixed_spectra
        
  
def get_test_dataset(inputs, inputs_mean_std, targets_mean_std):
    return SpectralTestDataset(
        spectra=inputs[:, None, :],
        concentrations=None,
        dtype=torch.float32,
        spectra_mean_std=inputs_mean_std,
        concentration_mean_std=targets_mean_std,
        combine_spectra_range=1.0,
        baseline_factor_bound=config["baseline_factor_bound"],
        baseline_period_lower_bound=config["baseline_period_lower_bound"],
        baseline_period_upper_bound=(config["baseline_period_lower_bound"] + config["baseline_period_span"]),
        augment_slope_std=config["augment_slope_std"],
        augment_intersept_std=0.0,
        rolling_bound=config["rolling_bound"],
        spectrum_rolling_sigma=0.01,
        augmentation_weight=0.1,
        original_datapoint_weight=1.,
    )

In [16]:
def get_ckpt_paths():
    output_dir = "/kaggle/working"
    output_files = sorted(os.listdir(output_dir))

    ckpt_paths = []
    for f in output_files:
        if "paper.pretrain.avg.fold" in f and "csv" not in f:
            ckpt_path = os.path.join(output_dir, f)
            ckpt_paths.append(ckpt_path)
            ckpt = torch.load(ckpt_path, weights_only=False)
            print(ckpt_path, ckpt["epoch"], ckpt["score"])
            
    return ckpt_paths

ckpt_paths = get_ckpt_paths()

/kaggle/working/paper.pretrain.avg.fold.0.pt 93 0.8796180235365756
/kaggle/working/paper.pretrain.avg.fold.1.pt 97 0.9127566862313433
/kaggle/working/paper.pretrain.avg.fold.2.pt 75 0.9185037460711425
/kaggle/working/paper.pretrain.avg.fold.3.pt 94 0.8696868817861656
/kaggle/working/paper.pretrain.avg.fold.4.pt 65 0.9160192436386044


In [22]:
import torch
from collections import OrderedDict

def average_state_dicts(state_dict_list):
    """
    Average the weights of several compatible PyTorch state_dicts.

    Parameters
    ----------
    state_dict_list : list[dict]
        List of state_dicts (e.g. from torch.load).
        All must have identical keys and compatible tensor shapes.

    Returns
    -------
    OrderedDict
        New state_dict whose tensors are the arithmetic mean
        of the corresponding tensors in the input state_dicts.
    """
    if not state_dict_list:
        raise ValueError("state_dict_list is empty")

    n = len(state_dict_list)
    # Ensure we don't modify the originals
    avg_sd = OrderedDict()

    # Iterate over every parameter/buffer key
    for k in state_dict_list[0]:
        # sum across models → float32 to avoid overflow on int types
        avg = sum(sd[k].float() for sd in state_dict_list) / n
        # cast back to original dtype if needed
        avg_sd[k] = avg.to(dtype=state_dict_list[0][k].dtype)

    return avg_sd


state_dicts = []
for cp in ckpt_paths:
    ckpt = get_ckpt(cp)
    state_dicts.append(ckpt["state_dict"])

avg_state_dict = average_state_dicts(state_dicts)
torch.save(avg_state_dict, "paper.finetune.avg.pt")

In [26]:
means = []
stds = []
for i in range(5):
    means.append(targets_mean_std[i][1:][0])
    stds.append(targets_mean_std[i][1:][1])

In [37]:
means = torch.stack(means).mean(dim=0)
stds = torch.stack(stds).mean(dim=0)

In [38]:
means, stds

(tensor([6.8749, 1.1728, 1.5785]), tensor([2.9148, 0.5509, 0.6642]))

In [39]:
def inference(ckpt_name, i):
    ckpt = get_ckpt(ckpt_name)
    
    test_inputs = get_test_data()
    test_ds = get_test_dataset(test_inputs, inputs_mean_std[i][1:], targets_mean_std[i][1:])
    test_dl = DataLoader(test_ds, batch_size=32)
    
    model = ReZeroNet(**config).to(device)
    model.load_state_dict(ckpt)#["state_dict"])
    model.eval()
    
    all_preds = []
    for inputs in test_dl:
        with torch.inference_mode():
            preds = model(inputs.cuda())
            preds = preds.double() 
            all_preds.append(cuda_to_np(preds))
            
    preds = np.concatenate(all_preds)
    
    #mus = targets_mean_std[i][1:][0]
    #sigmas = targets_mean_std[i][1:][1]
    mus = means
    sigmas = stds

    for i in range(3):
        preds[:, i] = reverse_zscore(preds[:, i], mus[i].numpy(), sigmas[i].numpy())
    
    return preds

preds = inference("paper.finetune.avg.pt", 2)
generate_csv(preds, "paper.finetune.avg.pretrain.avg.finetune.csv")
preds

array([[4.55467238, 0.97522789, 0.66487008],
       [6.00770376, 1.6246631 , 1.7874143 ],
       [5.58514936, 0.65663363, 1.16576816],
       [4.3727229 , 1.12202299, 0.55961518],
       [8.6422731 , 0.77807372, 1.07656401],
       [7.22980387, 1.45137016, 1.00378166],
       [7.51131046, 0.85546698, 0.42494893],
       [7.40632432, 1.60870025, 1.25100468],
       [7.42605724, 1.58711335, 1.08990877],
       [9.45828925, 0.78054841, 0.37888773],
       [8.51004121, 1.26016664, 1.38555842],
       [4.09113637, 1.32622821, 1.18116114],
       [4.94532741, 1.37489915, 1.39924715],
       [5.2825231 , 1.31122966, 1.97904221],
       [5.38963794, 1.33641478, 1.44609179],
       [7.78276895, 1.35065231, 0.91578834],
       [5.97056971, 1.28275809, 1.114799  ],
       [6.1743793 , 1.28473473, 1.50536339],
       [6.31448413, 1.44891708, 1.32722498],
       [5.24208025, 1.27367718, 1.32470422],
       [7.07865173, 1.23531133, 0.93651088],
       [6.41079666, 1.33530479, 1.33659076],
       [3.

In [ ]:
def ensemble_inference(ckpt_paths):
    test_inputs = get_test_data()
    all_preds = []

    for i, ckpt_path in enumerate(ckpt_paths):
        ckpt = get_ckpt(ckpt_path)
        
        model = ReZeroNet(**config).to(device)
        model.load_state_dict(ckpt["state_dict"])
        model.eval()

        test_ds = get_test_dataset(test_inputs, inputs_mean_std[i][1:], targets_mean_std[i][1:])
        test_dl = DataLoader(test_ds, batch_size=32)
        
        fold_preds = []
        for inputs in test_dl:
            with torch.inference_mode():
                preds = model(inputs.cuda())
                preds = cuda_to_np(preds.double())
                fold_preds.append(preds)
                
        fold_preds = np.concatenate(fold_preds)
        
        means = targets_mean_std[i][1:][0]
        stds = targets_mean_std[i][1:][1]
        for i in range(3):
            fold_preds[:, i] = reverse_zscore(fold_preds[:, i], means[i].numpy(), stds[i].numpy())
            
        all_preds.append(fold_preds)

    return np.mean(all_preds, axis=0)

preds = ensemble_inference(ckpt_paths)
generate_csv(preds, "paper.finetune.avg.pretrain.weights.ensemble.csv")
preds